# AIAP Week 6 - Building a Machine Learning Application

Welcome to the last week of our coursework! In this week, we will look at deploying a model we have built in previous weeks. Unlike past weeks, where the focus is on the development of a high-performing statistical model, we will look at how to properly use this model, to put it into a server, and let users use it without the need of opening up our Jupyter notebook.   
    
 It is important to note that because of the nature of this week's assignment, the skills required will be slightly different from prior weeks. As always, feel free to engage your peers or any of us if you would like more information. __For those who are new to serving code__, do take the time to understand the underlying concepts instead of copying code. __For those with prior experience__, you might want to focus on serialising a model, dockers, or exploring big data tools that are not commonly seen in the typical software stack.

Goals for the week:
1. To deploy an AI/Machine Learning application
2. To create a reproducible project
3. To prototype quickly and work smart i.e. look for existing open-source projects and improvise them based on your project needs
4. To have fun :)

Credits to Raimi Karim (batch 1 apprentice) for resources for the week.

## 1. Project Idea

Because the nature of this week's coursework is to build a simple machine learning app, the project will be more free form. Our approach is as follows: we will prepare a baseline problem statement with close hand-holding for people who have not had prior experience with software engineering to work on. If after approaching this toy example, you feel that you are capable of doing more, we warmly invite you to give it a try, and perhaps share with the class. If you feel that this is trivial, you may wish to attempt your own problem statement from the start.

<font color=darkblue>Our baseline project will be printed in this colour - dark blue. Jupyter Lab users may not be able to see this colour.</font>

Like any other app project, we begin with defining a good problem statement. Formulating a clear problem statement will help give clarity to the problem we are solving. A well defined problem statement now allows us to focus on engineering and model development later, which means less distraction and time wasted due to context switching.

One way to think about problem statements are through the '5W/1H' approach. Where we answer the following questions:  
  
- **Who** experiences this problem? What are their characteristics?  
- **What** is the actual problem?  
- **Why** does this app solve the problem?  
- **How** will it be implemented?  
- **When, where** should it be deployed? (not relevant for this week)

<font color=darkblue>For a baseline project, we will be attempting to build a fruit classifier, which attempts to classify 3 different types of fruits: apples, oranges and pears. Suppose we have someone who has difficulty identifying fruits. In that case, building this app will help him identify fruit appropriately.</font>

## 2. Workflow

We now move on to the actual workflow of training the model. The below is a standard set of steps for a mini-project. However, depending what what you do, you might need some variation, so modify them to your needs at your own discretion.

In [1]:
# All the imports
from google_images_download import google_images_download
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.models import Model, load_model
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import cv2
import os
import random
import gc
from PIL import Image

C:\Users\Raymond\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 2.1 Obtain data

We begin by collecting the data that we wish to train on. In this step, think more about the data you will need - how much would you need, and in what form should it be? In some cases, the problem we want to solve might already have a clean dataset to work on. In other cases, pre-trained models make our problem easier, and hence we only need a small fraction of a full-fledge training data.  

For problems where data might not be present, you might want to also think about how to collect and label data. <br/>

<font color=darkblue>For our case, we will begin by collecting images of each of the 3 fruits. We will use Google's image search to crawl a few pages of images for each label. Luckily, there are already tools out there to help you crawl of images - you can use this [python package](https://github.com/hardikvasa/google-images-download), or alternatively, `fastai` library has a [`download_images`](https://docs.fast.ai/vision.data.html#download_images) method.</font>

### For crawling of images from Google

In [2]:
# response = google_images_download.googleimagesdownload()   #class instantiation

# arguments = {"keywords":"red apple fruit,orange fruit,pear fruit","limit":200,"print_urls":True,"output_directory":"data/raw","chromedriver":"C:\\chromedriver_win32\\chromedriver.exe"}   #creating list of arguments
# paths = response.download(arguments)   #passing the arguments to the function
# print(paths)   #printing absolute paths of the downloaded images

### For renaming of all the files taken from google into a standard format for easy processing

In [3]:
# path = r'data\raw\orange fruit'
# files = os.listdir(path)
# for index, file in enumerate(files):
#    os.rename(os.path.join(path, file), os.path.join(path, 'orange_' + str(index) + '.jpg'))

In [4]:
# path = r'data\raw\pear fruit'
# files = os.listdir(path)
# for index, file in enumerate(files):
#    os.rename(os.path.join(path, file), os.path.join(path, 'pear_' + str(index) + '.jpg'))

In [5]:
# path = r'data\raw\red apple fruit'
# files = os.listdir(path)
# for index, file in enumerate(files):
#    os.rename(os.path.join(path, file), os.path.join(path, 'apple_' + str(index) + '.jpg'))

### For removing of files that are not in RGB

In [6]:
# Print out all the image type before deleting of files
path = r'data\raw\orange fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

orange_0.jpg: RGB
orange_1.jpg: RGB
orange_10.jpg: RGB
orange_100.jpg: RGB
orange_101.jpg: RGB
orange_102.jpg: RGB
orange_103.jpg: RGB
orange_104.jpg: RGB
orange_106.jpg: RGB
orange_107.jpg: RGB
orange_108.jpg: RGB
orange_109.jpg: RGB
orange_110.jpg: RGB
orange_111.jpg: RGB
orange_112.jpg: RGB
orange_113.jpg: RGB
orange_114.jpg: RGB
orange_115.jpg: RGB
orange_116.jpg: RGB
orange_117.jpg: RGB
orange_118.jpg: RGB
orange_119.jpg: RGB
orange_12.jpg: RGB
orange_120.jpg: RGB
orange_121.jpg: RGB
orange_122.jpg: RGB
orange_123.jpg: RGB
orange_124.jpg: RGB
orange_125.jpg: RGB
orange_126.jpg: RGB
orange_128.jpg: RGB
orange_129.jpg: RGB
orange_13.jpg: RGB
orange_131.jpg: RGB
orange_132.jpg: RGB
orange_133.jpg: RGB
orange_134.jpg: RGB
orange_135.jpg: RGB
orange_136.jpg: RGB
orange_137.jpg: RGB
orange_139.jpg: RGB
orange_141.jpg: RGB
orange_142.jpg: RGB
orange_143.jpg: RGB
orange_144.jpg: RGB
orange_146.jpg: RGB
orange_147.jpg: RGB
orange_148.jpg: RGB
orange_149.jpg: RGB
orange_15.jpg: RGB
orange_1

In [7]:
# Delete files that are not in RGB
path = r'data\raw\orange fruit'
files = os.listdir(path)
for file in files:
    img = Image.open(path + "\\" + file)
    if (img.mode != "RGB"):
        img.close()
        os.remove(path + "\\" + file)

In [8]:
# Print out the files that are left
path = r'data\raw\orange fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

orange_0.jpg: RGB
orange_1.jpg: RGB
orange_10.jpg: RGB
orange_100.jpg: RGB
orange_101.jpg: RGB
orange_102.jpg: RGB
orange_103.jpg: RGB
orange_104.jpg: RGB
orange_106.jpg: RGB
orange_107.jpg: RGB
orange_108.jpg: RGB
orange_109.jpg: RGB
orange_110.jpg: RGB
orange_111.jpg: RGB
orange_112.jpg: RGB
orange_113.jpg: RGB
orange_114.jpg: RGB
orange_115.jpg: RGB
orange_116.jpg: RGB
orange_117.jpg: RGB
orange_118.jpg: RGB
orange_119.jpg: RGB
orange_12.jpg: RGB
orange_120.jpg: RGB
orange_121.jpg: RGB
orange_122.jpg: RGB
orange_123.jpg: RGB
orange_124.jpg: RGB
orange_125.jpg: RGB
orange_126.jpg: RGB
orange_128.jpg: RGB
orange_129.jpg: RGB
orange_13.jpg: RGB
orange_131.jpg: RGB
orange_132.jpg: RGB
orange_133.jpg: RGB
orange_134.jpg: RGB
orange_135.jpg: RGB
orange_136.jpg: RGB
orange_137.jpg: RGB
orange_139.jpg: RGB
orange_141.jpg: RGB
orange_142.jpg: RGB
orange_143.jpg: RGB
orange_144.jpg: RGB
orange_146.jpg: RGB
orange_147.jpg: RGB
orange_148.jpg: RGB
orange_149.jpg: RGB
orange_15.jpg: RGB
orange_1

In [9]:
# Print out all the image type before deleting of files
path = r'data\raw\pear fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

pear_0.jpg: RGB
pear_1.jpg: RGB
pear_101.jpg: RGB
pear_104.jpg: RGB
pear_105.jpg: RGB
pear_106.jpg: RGB
pear_107.jpg: RGB
pear_108.jpg: RGB
pear_109.jpg: RGB
pear_11.jpg: RGB
pear_110.jpg: RGB
pear_111.jpg: RGB
pear_112.jpg: RGB
pear_113.jpg: RGB
pear_114.jpg: RGB
pear_115.jpg: RGB
pear_116.jpg: RGB
pear_117.jpg: RGB
pear_118.jpg: RGB
pear_119.jpg: RGB
pear_120.jpg: RGB
pear_121.jpg: RGB
pear_122.jpg: RGB
pear_123.jpg: RGB
pear_124.jpg: RGB
pear_125.jpg: RGB
pear_126.jpg: RGB
pear_127.jpg: RGB
pear_128.jpg: RGB
pear_129.jpg: RGB
pear_13.jpg: RGB
pear_130.jpg: RGB
pear_131.jpg: RGB
pear_132.jpg: RGB
pear_133.jpg: RGB
pear_134.jpg: RGB
pear_135.jpg: RGB
pear_136.jpg: RGB
pear_137.jpg: RGB
pear_138.jpg: RGB
pear_139.jpg: RGB
pear_140.jpg: RGB
pear_141.jpg: RGB
pear_142.jpg: RGB
pear_144.jpg: RGB
pear_145.jpg: RGB
pear_146.jpg: RGB
pear_149.jpg: RGB
pear_150.jpg: RGB
pear_151.jpg: RGB
pear_152.jpg: RGB
pear_153.jpg: RGB
pear_154.jpg: RGB
pear_155.jpg: RGB
pear_156.jpg: RGB
pear_157.jpg: RG

In [10]:
# Delete files that are not in RGB
path = r'data\raw\pear fruit'
files = os.listdir(path)
for file in files:
    img = Image.open(path + "\\" + file)
    if (img.mode != "RGB"):
        img.close()
        os.remove(path + "\\" + file)

In [11]:
# Print out the files that are left
path = r'data\raw\pear fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

pear_0.jpg: RGB
pear_1.jpg: RGB
pear_101.jpg: RGB
pear_104.jpg: RGB
pear_105.jpg: RGB
pear_106.jpg: RGB
pear_107.jpg: RGB
pear_108.jpg: RGB
pear_109.jpg: RGB
pear_11.jpg: RGB
pear_110.jpg: RGB
pear_111.jpg: RGB
pear_112.jpg: RGB
pear_113.jpg: RGB
pear_114.jpg: RGB
pear_115.jpg: RGB
pear_116.jpg: RGB
pear_117.jpg: RGB
pear_118.jpg: RGB
pear_119.jpg: RGB
pear_120.jpg: RGB
pear_121.jpg: RGB
pear_122.jpg: RGB
pear_123.jpg: RGB
pear_124.jpg: RGB
pear_125.jpg: RGB
pear_126.jpg: RGB
pear_127.jpg: RGB
pear_128.jpg: RGB
pear_129.jpg: RGB
pear_13.jpg: RGB
pear_130.jpg: RGB
pear_131.jpg: RGB
pear_132.jpg: RGB
pear_133.jpg: RGB
pear_134.jpg: RGB
pear_135.jpg: RGB
pear_136.jpg: RGB
pear_137.jpg: RGB
pear_138.jpg: RGB
pear_139.jpg: RGB
pear_140.jpg: RGB
pear_141.jpg: RGB
pear_142.jpg: RGB
pear_144.jpg: RGB
pear_145.jpg: RGB
pear_146.jpg: RGB
pear_149.jpg: RGB
pear_150.jpg: RGB
pear_151.jpg: RGB
pear_152.jpg: RGB
pear_153.jpg: RGB
pear_154.jpg: RGB
pear_155.jpg: RGB
pear_156.jpg: RGB
pear_157.jpg: RG

In [12]:
# Print out all the image type before deleting of files
path = r'data\raw\red apple fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

apple_0.jpg: RGB
apple_10.jpg: RGB
apple_100.jpg: RGB
apple_101.jpg: RGB
apple_103.jpg: RGB
apple_104.jpg: RGB
apple_105.jpg: RGB
apple_106.jpg: RGB
apple_107.jpg: RGB
apple_108.jpg: RGB
apple_109.jpg: RGB
apple_11.jpg: RGB
apple_111.jpg: RGB
apple_112.jpg: RGB
apple_113.jpg: RGB
apple_115.jpg: RGB
apple_116.jpg: RGB
apple_117.jpg: RGB
apple_118.jpg: RGB
apple_119.jpg: RGB
apple_12.jpg: RGB
apple_120.jpg: RGB
apple_121.jpg: RGB
apple_122.jpg: RGB
apple_123.jpg: RGB
apple_124.jpg: RGB
apple_126.jpg: RGB
apple_127.jpg: RGB
apple_128.jpg: RGB
apple_129.jpg: RGB
apple_13.jpg: RGB
apple_130.jpg: RGB
apple_131.jpg: RGB
apple_132.jpg: RGB
apple_133.jpg: RGB
apple_134.jpg: RGB
apple_135.jpg: RGB
apple_136.jpg: RGB
apple_138.jpg: RGB
apple_139.jpg: RGB
apple_14.jpg: RGB
apple_140.jpg: RGB
apple_141.jpg: RGB
apple_143.jpg: RGB
apple_144.jpg: RGB
apple_145.jpg: RGB
apple_146.jpg: RGB
apple_147.jpg: RGB
apple_148.jpg: RGB
apple_149.jpg: RGB
apple_150.jpg: RGB
apple_151.jpg: RGB
apple_152.jpg: RGB


In [13]:
# Delete files that are not in RGB
path = r'data\raw\red apple fruit'
files = os.listdir(path)
for file in files:
    img = Image.open(path + "\\" + file)
    if (img.mode != "RGB"):
        img.close()
        os.remove(path + "\\" + file)

In [14]:
# Print out the files that are left
path = r'data\raw\red apple fruit'
files = os.listdir(path)
for index, file in enumerate(files):
    img = Image.open(path + "\\" + file)
    print(file + ": " + img.mode)

apple_0.jpg: RGB
apple_10.jpg: RGB
apple_100.jpg: RGB
apple_101.jpg: RGB
apple_103.jpg: RGB
apple_104.jpg: RGB
apple_105.jpg: RGB
apple_106.jpg: RGB
apple_107.jpg: RGB
apple_108.jpg: RGB
apple_109.jpg: RGB
apple_11.jpg: RGB
apple_111.jpg: RGB
apple_112.jpg: RGB
apple_113.jpg: RGB
apple_115.jpg: RGB
apple_116.jpg: RGB
apple_117.jpg: RGB
apple_118.jpg: RGB
apple_119.jpg: RGB
apple_12.jpg: RGB
apple_120.jpg: RGB
apple_121.jpg: RGB
apple_122.jpg: RGB
apple_123.jpg: RGB
apple_124.jpg: RGB
apple_126.jpg: RGB
apple_127.jpg: RGB
apple_128.jpg: RGB
apple_129.jpg: RGB
apple_13.jpg: RGB
apple_130.jpg: RGB
apple_131.jpg: RGB
apple_132.jpg: RGB
apple_133.jpg: RGB
apple_134.jpg: RGB
apple_135.jpg: RGB
apple_136.jpg: RGB
apple_138.jpg: RGB
apple_139.jpg: RGB
apple_14.jpg: RGB
apple_140.jpg: RGB
apple_141.jpg: RGB
apple_143.jpg: RGB
apple_144.jpg: RGB
apple_145.jpg: RGB
apple_146.jpg: RGB
apple_147.jpg: RGB
apple_148.jpg: RGB
apple_149.jpg: RGB
apple_150.jpg: RGB
apple_151.jpg: RGB
apple_152.jpg: RGB


### 2.2 Preprocess data

After getting the data, the next step is to see if our data is clean, and can be fed into the model.

Check if our data is clean involves checking for null values, extreme outliers, consistency of file names etc. This is dependent on the nature of your dataset. Sometimes it might also help to visualise your data.

We would also like to split our data into train-validation-test sets to track if our trained model later performs as well on the validation/test set as it did on our training set.

<font color=darkblue>In our case, we will like to split this dataset into train and test sets, using appropriate data loaders/generators to create labels of 0, 1 and 2 for the fruits (or otherwise appropriate). Using the appropriate library of your choice, preprocess the images with the right preprocessing methods - resizing into 224px, then doing a center crop, normalizing colour values (you can just divide by 255), and finally turning each image into a 3x244x244 tensor. You may also do additional augments if you please.</font>

#### Code for processing a single image to be passed into the model for prediction.

In [15]:
img_path = r'data\raw\orange fruit\orange_23.jpg'
img = image.load_img(img_path, target_size=(224, 224))
sample = image.img_to_array(img)
print(sample.shape)
sample = np.expand_dims(sample, axis=0)
print(sample.shape)
sample = preprocess_input(sample)
print(sample.shape)

(224, 224, 3)
(1, 224, 224, 3)
(1, 224, 224, 3)


#### Using Keras ImageDataGenerator to directly process the images in the folder for training

In [16]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                rotation_range=15,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                shear_range=0.01,
                                zoom_range=[0.9, 1.25],
                                horizontal_flip=True,
                                vertical_flip=False,
                                fill_mode='reflect',
                                brightness_range=[0.5, 1.5],
                                validation_split=0.2)

train_generator = train_datagen.flow_from_directory('./data/raw/',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('./data/raw/',
                                                    target_size=(224,224),
                                                    color_mode='rgb',
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='validation')

Found 394 images belonging to 3 classes.
Found 97 images belonging to 3 classes.


#### Attempt at manually processing the images (Please ignore. Not fully completed)

In [17]:
# apple_dir = 'data/raw/red apple fruit'
# orange_dir = 'data/raw/orange fruit'
# pear_dir = 'data/raw/pear fruit'

# train_apples = ['data/raw/red apple fruit/{}'.format(i) for i in os.listdir(apple_dir) if 'apple' in i]
# train_oranges = ['data/raw/orange fruit/{}'.format(i) for i in os.listdir(orange_dir) if 'orange' in i]
# train_pears = ['data/raw/pear fruit/{}'.format(i) for i in os.listdir(pear_dir) if 'pear' in i]

# train_imgs = train_apples + train_oranges + train_pears
# random.shuffle(train_imgs)

# del train_apples
# del train_oranges
# del train_pears
# gc.collect()

In [18]:
# for ima in train_imgs[0:3]:
#     img = mpimg.imread(ima)
#     imgplot = plt.imshow(img)
#     plt.show()

In [19]:
# nrows = 224
# ncolumns = 224
# channels = 3

In [20]:
# def read_and_process_image(list_of_images):
#     X = []
#     y = []
    
#     for index, image in enumerate(list_of_images):
#         try:
#             X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (nrows,ncolumns), interpolation=cv2.INTER_CUBIC))
#             if 'apple' in image:
#                 y.append(0)
#             elif 'orange' in image:
#                 y.append(1)
#             elif 'pear' in image:
#                 y.append(2)
#         except Exception:
#             continue
#     return X, y

In [21]:
# X, y = read_and_process_image(train_imgs)

In [22]:
# plt.figure(figsize=(20,10))
# columns = 5
# for i in range(columns):
#     plt.subplot(5 / columns + 1, columns, i + 1)
#     plt.imshow(X[i])

### 2.3 Train model

Train your model locally (on your computer) or in Google Colab. Once done, run a few tests on the test set, and see if it performs as well as in the training set. If you are not satisfied, you can retrain your model. Finally, export your model out with the appropriate format, and download the weights for future use. In most cases, you should look to run a transfer learning model on an already optimised model. For image models, you can try something like resnet50. There have been some discussion about transfer learning in text models too - models such as BERT and ELMo... yes we agree that the names are ridiculous.

<font color=darkblue>In our case, let's begin with a pretrained resnet50 model - ample code examples should be available for both torchvision and Keras. Freeze all layers except the last, and replace that with a linear layer of output size 3, fed through a softmax loss function. Be weary of overfitting when you train your model.</font>

In [23]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = base_model.output
# x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
# x = Dense(1024,activation='relu')(x)
# x = Dense(1024,activation='relu')(x)
# x = Dense(512,activation='relu')(x)
preds = Dense(3,activation='softmax')(x) #final layer with softmax activation

C:\Users\Raymond\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [24]:
model = Model(inputs=base_model.input, outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [26]:
# for layer in model.layers:
#     layer.trainable = False

for layer in model.layers[:-1]:
    layer.trainable = False

In [27]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_generator.n // train_generator.batch_size
model.fit_generator(generator = train_generator,
                    steps_per_epoch = step_size_train,
                    validation_data = validation_generator,
                    validation_steps = validation_generator.n // validation_generator.batch_size,
                    epochs = 5)

Epoch 1/5
12/12 [==============================] - 21s 2s/step - loss: 1.8535 - acc: 0.7604 - val_loss: 1.8899 - val_acc: 0.8438
Epoch 2/5
12/12 [==============================] - 13s 1s/step - loss: 0.8599 - acc: 0.8926 - val_loss: 0.5915 - val_acc: 0.9385
Epoch 3/5
12/12 [==============================] - 13s 1s/step - loss: 0.2739 - acc: 0.9633 - val_loss: 0.9068 - val_acc: 0.9077
Epoch 4/5
12/12 [==============================] - 13s 1s/step - loss: 0.3847 - acc: 0.9634 - val_loss: 0.3907 - val_acc: 0.9385
Epoch 5/5
12/12 [==============================] - 15s 1s/step - loss: 0.2727 - acc: 0.9660 - val_loss: 0.5755 - val_acc: 0.9479


#### Keras ImageDataGenerator flow_from_directory process the classes in alphabetical order of the older name.
Index 0: Orange
<br/>
Index 1: Pear
<br/>
Index 2: Apple

In [28]:
labels = {0:"Orange", 1:"Pear", 2:"Apple"}
print(labels)

{0: 'Orange', 1: 'Pear', 2: 'Apple'}


In [29]:
preds = model.predict(sample)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# print(decode_predictions(preds, top=1)[0][0][1])
labels.get(preds.argmax())

'Orange'

#### Saving of the model for use in the website directly

In [30]:
model.save_weights('fruit_model_weights.h5')
model.save('fruit_model.h5')

In [31]:
model_reloaded = load_model('fruit_model.h5')
repreds = model_reloaded.predict(sample)
labels.get(repreds.argmax())

'Orange'

### 2.4 Deploy model

You can start with a simple localhosted [Python Flask](http://flask.pocoo.org) app for the deployment of your model. In this Flask app, provide an HTML endpoint that allows a user to submit a datapoint, and provide a prediction. Ensure that a user is able to make calls to your API by providing an input. A simple way to test this would be through the Postman application.

<font color=darkblue>We will use the PIL package to help process the image given into the appropriate format. Your endpoint should accept a 224x224 image, which PIL can convert into a numpy array to be ingested as a tensor into the model.

Write a `@app.route` to take in a request from a user, and provide a prediction response to the user. For those who are new to applications, take some time to understand how web servers are designed, and specifically, what are routes  in the context of a Flask app.</font>

For people who are more advanced, you could also deploy your model in the following environments:
- **Server**
  - Local
  - Self-hosted (WSGI, Nginx)
  - App Engine
  
- **Docker**
- **Front-end browser-based**
  - tensorflow.js + GitHub Pages
- **Mobile (Android)** <br>
  - Package as an APK for distribution

## 3. A Little Bit More

In this next section, we will look at some other bits of application development that could be useful to know.

### 3.1 Unit Testing, Integration Testing

In the space of software engineering, complex operations happening within apps, in conjunction with multiple engineers working on multiple areas, make application development a hairy and unstable process. To mitigate the risks of bugs appearing in production environments, we write thorough tests that, to the best of our ability, ensure the accuracy and stability of our code. 

We will not go into detail into this section as this lies in the domain of software engineering, but understand that in production environments, testing is key to stability of any app. In addition, we use approaches like test-driven development (TDD), as well as a mix of tests at different levels.

It is interesting to note the inherent probabalistic nature of machine learning models - we are not going to get the right predictions every time. How do we ensure that our tests will ensure the correctness of our code in the probabalistic, flaky environment of machine learning? We don't exactly have a gold standard answer to this question right now.

### 3.2 Documentation (Important - Submission Details)

Documentation is extremely important to ensure that when we pass our code on to another analyst or engineer, they will know how to work with and modify the code for their own use. Especially given that AI Singapore needs highly reusable code and exchange of information between teams, we would like to set high standards for documentation.

For this assignment, we would expect a basic level of documentation written in a `README.md` in your repository. This document should discuss what your app would do, and give basic instructions - lines of bash commands - that can guide a user to run your code base on his computer. The right way to do this is to ask a partner to try running your code from the GitHub repository. If it runs with your instructions, you're good to go.

[Here's a good, simple README.md file.](https://github.com/berlotto/flask-app-template)

__Submission__: to submit, create a NEW REPOSITORY this week inside [this new project folder](https://bitbucket.ai3.aisingapore.org:9443/projects/ABA/) and push your code instead. Be sure to include a README.md as well as a [.gitignore](https://www.gitignore.io/) (as discussed by Deepan a few weeks ago). Please __do not__ upload large datasets or model weights into the repository - we have limited server space!

## 4. Examples

These are some ideas to get you started:


**Image**

  - Classify flowers: https://www.tensorflow.org/hub/tutorials/image_retraining
  - Classify pictures of food for mobile users that helps them to easily attach hashtags for their Instagram post.
  - Create your home security camera that detects faces of your family members using Raspberry Pi and grants access.
  - Classify if a picture is an item that can be recycled or not. 
  - Recognise mathematical equations using Optical Character Recognition.
  - Singlish meme: Automatically caption pictures.
  - Style transfer: Generate an image of a utopia given an input.

**Text**
  - Generate text https://www.tensorflow.org/tutorials/sequences/text_generation
  - Summarise a code in 1 paragraph.
  - Using POS tagging, summarise a research paper in 1 paragraph.
  - Use NLP to generate a Jupyter notebook exercise for AIAP for the next week.
  - Cluster AI Singapore's web pages into meaningful categories.
  - Generate/classify fake news in Singapore.
  - A Singlish aunty-chatbot for buying groceries.

**Audio**
  - Classify sounds https://www.tensorflow.org/tutorials/sequences/audio_recognition
  - Classify between the different Chinese dialects.
  - Speech-to-Text application that identifies the main keywords and googles them.
  - Transpose a music from major to minor scale using recurrent neural networks.

**Video**
  - A better push-up/sit-ups counter (oops).
  - Dance-Dance revolution.
  - Control a character in a game using pose recognition.